In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
import random 
import pickle
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import seaborn as sns
# from ucimlrepo import fetch_ucirepo 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

from data.data_loader import EPCDataset, PowerWeatherDatasetWithSeason, PowerWeatherDataset, SingleTermDataset, MultiTermDataset
from models.lstm import LSTMModel
from models.lstm_attention import LSTMWithAttention, BiLSTMWithAttention
from models.gru import GRUModel
from models.utils import create_model, train_and_evaluate, load_model, evaluate_model, evaluate_ensemble

from explainers.utils import get_explainer
# from explainers.lime import LimeExplainer
# from explainers.shap import ShapExplainer
# from explainers.attention import AttentionExplainer
# from explainers.grad_cam import GradCAMExplainer
# from explainers.lrp import LRPExplainer

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
features_for_1h = [
    'Global_active_power',
    'Global_intensity',
    'Sub_metering_1', 
    'Sub_metering_2', 
    'Sub_metering_3', 
    'Temperature',	
    'Humidity',	
]

features_for_6h = [
    'Global_active_power',
    'Global_intensity',
    'Sub_metering_1', 
    'Sub_metering_2', 
    'Sub_metering_3', 
    'Temp_Min',	'Temp_Max', 'Temp_Avg',	'Temp_Range',
    'Humidity_Min',	'Humidity_Max',	'Humidity_Avg',	'Humidity_Range'
]


file_path_for_1h = 'data/final_data.csv'
file_path_for_6h = 'data/final_data_per_6hr_with_avg_range.csv'


long_term_length = 365 * 4
long_term_pred_length = 256 # ?64, 128, 256

short_term_length = 30 * 24 # 1시간 단위 한달 데이터
short_term_pred_length = 7 * 24 # 하루치 데이터 예측

In [4]:
dataset_1h = EPCDataset(
    file_path=file_path_for_1h,
    sequence_length=short_term_length,  # Use short-term length for 1-hour data
    prediction_length=short_term_pred_length,
    target_features=features_for_1h
)

train_short, train_targets_short, eval_short, eval_targets_short = dataset_1h.load_data()

print("1-hour data sequences generated:")
print(f"Train sequences shape: {train_short.shape}")
print(f"Train targets shape: {train_targets_short.shape}")
print(f"Eval sequences shape: {eval_short.shape}")
print(f"Eval targets shape: {eval_targets_short.shape}")

# Generate sequences for 6-hour data
print("\nProcessing 6-hour data...")
dataset_6h = EPCDataset(
    file_path=file_path_for_6h,
    sequence_length=long_term_length,  # Use long-term length for 6-hour data
    prediction_length=long_term_pred_length,
    target_features=features_for_6h
)

train_long, train_targets_long, eval_long, eval_targets_long = dataset_6h.load_data()

# #  1시간 1년
# dataset_long = EPCDataset(
#     file_path=file_path_for_1h,
#     sequence_length=long_term_length,  # Use long-term length for 6-hour data
#     prediction_length=long_term_pred_length,
#     target_features=features_for_1h
# )

# train_long, train_targets_long, eval_long, eval_targets_long = dataset_long.load_data()


print("6-hour data sequences generated:")
print(f"Train sequences shape: {train_long.shape}")
print(f"Train targets shape: {train_targets_long.shape}")
print(f"Eval sequences shape: {eval_long.shape}")
print(f"Eval targets shape: {eval_targets_long.shape}")

1-hour data sequences generated:
Train sequences shape: torch.Size([23810, 720, 13])
Train targets shape: torch.Size([23810, 168])
Eval sequences shape: torch.Size([5953, 720, 13])
Eval targets shape: torch.Size([5953, 168])

Processing 6-hour data...
6-hour data sequences generated:
Train sequences shape: torch.Size([2713, 1460, 19])
Train targets shape: torch.Size([2713, 256])
Eval sequences shape: torch.Size([679, 1460, 19])
Eval targets shape: torch.Size([679, 256])


In [5]:
from models.lstm import LSTMModel
from models.gru import GRUModel
from models.cnnlstm import CNNLSTMModel
from models.lstm_attention import AttentionLSTMModel, LSTMWithAttention
from models.ls_cnnlstm import LongShortCNNLSTM
from models.ls_cnnlstm_attention import LongShortCNNLSTMWithAttention

def create_model2(model_name, input_size, hidden_size, num_layers, output_size, dropout=0.2, long_term_length=None, short_term_length=None):
    ls_key = 'short'
    
    if model_name == 'GRU':
        return GRUModel(input_size[ls_key], hidden_size, num_layers, output_size[ls_key], dropout)
    elif model_name == 'LSTM':
        return LSTMModel(input_size[ls_key], hidden_size, num_layers, output_size[ls_key], dropout)
    elif model_name == 'LSTM-Att':
        return LSTMWithAttention(input_size[ls_key], hidden_size, num_layers, output_size[ls_key], dropout)
    elif model_name == 'CNNLSTM':
        return CNNLSTMModel(input_size[ls_key], hidden_size, num_layers, output_size[ls_key], dropout)
    elif model_name == 'LS_CNNLSTM': 
        return LongShortCNNLSTM(
            long_input_size=input_size['long'],
            short_input_size=input_size['short'],
            hidden_size=hidden_size,
            num_layers=num_layers,
            long_output_size=output_size['long'],
            short_output_size=output_size['short'],
            long_term_length=long_term_length,
            short_term_length=short_term_length,
            dropout=dropout
        )
    elif model_name == 'LS_CNNLSTM_Att': 
        return LongShortCNNLSTMWithAttention(
            input_dim_long=input_size['long'],
            input_dim_short=input_size['short'],
            hidden_dim=hidden_size,
            long_output_dim=output_size['long'],
            output_dim=output_size['short'], 
            seq_len_long=long_term_length, 
            seq_len_short=short_term_length)
    else:
        raise ValueError(f"Model {model_name} is not recognized.")

In [6]:
class LS_Loss(nn.Module):
    def __init__(self, alpha=1.0, beta=0.3, gamma=0.0):
        super(LS_Loss, self).__init__()
        self.alpha = alpha  # Weight for Short-term Loss
        self.beta = beta    # Weight for Long-term Loss
        self.gamma = gamma  # Weight for Cross-Modality Consistency Loss
        self.mse_loss = nn.MSELoss()

    def forward(self, output_long, target_long, output_short, target_short):
        # Short-term Loss
        loss_short = self.mse_loss(output_short, target_short)

        # Long-term Loss
        loss_long = self.mse_loss(output_long, target_long)

        # Optional: Cross-Modality Consistency Loss
        consistency_loss = 0.0
        if self.gamma > 0:
            long_mean = torch.mean(output_long, dim=1, keepdim=True)
            short_mean = torch.mean(output_short, dim=1, keepdim=True)
            consistency_loss = torch.mean((long_mean - short_mean) ** 2)

        # Combined Loss
        total_loss = self.alpha * loss_short + self.beta * loss_long + self.gamma * consistency_loss
        return total_loss

class SimpleMSELoss(nn.Module):
    def __init__(self, alpha=1.0, beta=0.1):
        super(SimpleMSELoss, self).__init__()
        self.alpha = alpha  # Weight for Short-term Loss
        self.beta = beta    # Weight for Long-term Loss
        self.mse_loss = nn.MSELoss()

    def forward(self, output_long, target_long, output_short, target_short):
        # Short-term Loss
        loss_short = self.mse_loss(output_short, target_short)

        # Long-term Loss
        loss_long = self.mse_loss(output_long, target_long)

        # Combined Loss
        total_loss = self.alpha * loss_short + self.beta * loss_long
        return total_loss

def oversample_data(long_data, short_data, long_targets, short_targets):
    """
    Oversample short-term data to match the size of long-term data.
    """
    # Check the smaller dataset
    if len(short_data) < len(long_data):
        # Calculate the oversample factor and residual
        oversample_factor = len(long_data) // len(short_data)
        residual_samples = len(long_data) % len(short_data)

        # Oversample short data
        short_data = torch.cat([short_data] * oversample_factor + [short_data[:residual_samples]], dim=0)
        short_targets = torch.cat([short_targets] * oversample_factor + [short_targets[:residual_samples]], dim=0)
    elif len(long_data) < len(short_data):
        # Calculate the oversample factor and residual for long data
        oversample_factor = len(short_data) // len(long_data)
        residual_samples = len(short_data) % len(long_data)

        # Oversample long data
        long_data = torch.cat([long_data] * oversample_factor + [long_data[:residual_samples]], dim=0)
        long_targets = torch.cat([long_targets] * oversample_factor + [long_targets[:residual_samples]], dim=0)
    
    # Ensure shapes match after oversampling
    assert len(long_data) == len(short_data), "Oversampling failed to match dataset sizes!"
    assert len(long_targets) == len(short_targets), "Oversampling failed to match target sizes!"

    return long_data, short_data, long_targets, short_targets
    

def train_and_evaluate3(model, model_name, train_data, train_targets, eval_data, eval_targets, model_path, num_epochs=100, batch_size=64, learning_rate=0.001, patience=5, oversample_eval=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_long, train_short = train_data
    train_targets_long, train_targets_short = train_targets

    eval_long, eval_short = eval_data
    eval_targets_long, eval_targets_short = eval_targets

    # Oversample training data
    train_long, train_short, train_targets_long, train_targets_short = oversample_data(
        train_long, train_short, train_targets_long, train_targets_short
    )
    print(f"After Oversampling:")
    print(f"Train Long-term shape: {train_long.shape}")
    print(f"Train Short-term shape: {train_short.shape}")

    # Optionally oversample evaluation data
    if oversample_eval:
        eval_long, eval_short, eval_targets_long, eval_targets_short = oversample_data(
            eval_long, eval_short, eval_targets_long, eval_targets_short
        )

    # Adjust evaluation data sizes if oversample is not applied
    if not oversample_eval:
        min_eval_size = min(len(eval_long), len(eval_short))
        eval_long = eval_long[:min_eval_size]
        eval_short = eval_short[:min_eval_size]
        eval_targets_long = eval_targets_long[:min_eval_size]
        eval_targets_short = eval_targets_short[:min_eval_size]

    # DataLoaders
    train_dataset = torch.utils.data.TensorDataset(train_long, train_short, train_targets_long, train_targets_short)
    eval_dataset = torch.utils.data.TensorDataset(eval_long, eval_short, eval_targets_long, eval_targets_short)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(dataset=eval_dataset, batch_size=batch_size, shuffle=False)

    # Loss function and optimizer
    # criterion = nn.MSELoss()
    criterion = SimpleMSELoss(alpha=0.3, beta=1.0)
    # criterion = LS_Loss(alpha=1.0, beta=0.3, gamma=0.2, lambda_=0.1, eta=0.01)
    
    # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0

        for long_batch, short_batch, target_long, target_short in train_loader:
            long_batch, short_batch = long_batch.to(device), short_batch.to(device)
            target_long, target_short = target_long.to(device), target_short.to(device)

            if 'Att' in model_name:
                output_long, output_short, attention_weights = model(long_batch, short_batch)
            else:
                output_long, output_short = model(long_batch, short_batch)
            
            # loss = criterion(output_long, target_long) + criterion(output_short, target_short)
            loss = criterion(output_long, target_long, output_short, target_short)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        eval_loss = 0.0
        with torch.no_grad():
            for long_batch, short_batch, target_long, target_short in eval_loader:
                long_batch, short_batch = long_batch.to(device), short_batch.to(device)
                target_long, target_short = target_long.to(device), target_short.to(device)
                    
                if 'Att' in model_name:
                    output_long, output_short, attention_weights = model(long_batch, short_batch)
                else:
                    output_long, output_short = model(long_batch, short_batch)
                
                # loss = criterion(output_long, target_long) + criterion(output_short, target_short)
                loss = criterion(output_long, target_long, output_short, target_short)
                
                eval_loss += loss.item()

        eval_loss /= len(eval_loader)
        scheduler.step(eval_loss)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {eval_loss:.4f}")

        criterion.alpha = min(1.0, criterion.alpha + 0.05)  # Increase alpha
        criterion.beta = max(0.1, criterion.beta - 0.05)   # Decrease beta
        
        # Early stopping
        if eval_loss < best_val_loss:
            best_val_loss = eval_loss
            torch.save(model.state_dict(), model_path)
            print(f"Validation loss improved. Model saved at epoch {epoch+1}")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

In [7]:
def train_and_evaluate2(model, model_name, train_sequences, train_targets, 
                       eval_sequences, eval_targets, model_path, num_epochs=100, 
                       batch_size=64, learning_rate=0.001, patience=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Data loader
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_sequences, dtype=torch.float32), torch.tensor(train_targets, dtype=torch.float32))
    eval_dataset = torch.utils.data.TensorDataset(torch.tensor(eval_sequences, dtype=torch.float32), torch.tensor(eval_targets, dtype=torch.float32))
    
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset, batch_size=batch_size, shuffle=False)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #, weight_decay=1e-5)

    best_val_loss = np.inf
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # tqdm을 사용하여 학습 진행도 표시
        with tqdm(train_loader, unit="batch") as tepoch:
            for sequences_batch, targets_batch in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
                
                # Move batch to GPU
                sequences_batch, targets_batch = sequences_batch.to(device), targets_batch.to(device)
                
                # Forward pass
                if 'Att' in model_name:
                    outputs, _ = model(sequences_batch) 
                else:
                    outputs = model(sequences_batch)

                loss = criterion(outputs.squeeze(), targets_batch)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # 현재 배치 손실을 누적
                running_loss += loss.item()
                tepoch.set_postfix(loss=loss.item())

        # Epoch 단위로 평균 손실 계산
        train_loss = running_loss / len(train_loader)

        # Evaluation on the validation set
        model.eval()
        eval_loss = 0.0
        with torch.no_grad():
            for sequences_batch, targets_batch in eval_loader:
                # Move batch to GPU
                sequences_batch, targets_batch = sequences_batch.to(device), targets_batch.to(device)

                if 'Att' in model_name:
                    val_outputs, _ = model(sequences_batch) 
                else:
                    val_outputs = model(sequences_batch)
                loss = criterion(val_outputs.squeeze(), targets_batch)
                eval_loss += loss.item()

        eval_loss /= len(eval_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {eval_loss:.7f}')

        # Check if the validation loss improved
        if eval_loss < best_val_loss:
            best_val_loss = eval_loss
            torch.save(model.state_dict(), model_path)
            print(f"Validation loss improved. Model saved at epoch {epoch+1}")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early stopping
        if epochs_no_improve >= patience:
            print(f"Early stopping applied at epoch {epoch+1}")
            break

In [8]:
def load_trained_model(params=None):
    if params is None:
        return

    # Extract parameters
    model_name = params['model_name']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    dropout = params['dropout']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    patience = params['patience']

    # Determine input size and output size
    input_size = {
        'long': len(features_for_6h)+6,
        'short': len(features_for_1h)+6
    }
    output_size = {
        'long': long_term_pred_length,
        'short': short_term_pred_length
    }

    # Model 생성 
    model = create_model2(
        model_name=model_name,
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        output_size=output_size,
        dropout=dropout, 
        long_term_length=long_term_length, 
        short_term_length=short_term_length
    )

    if 'LS_CNNLSTM' in model_name:
        model_path = './trained_models/{}_long_{}_short_{}_{}_{}_{}_ab.pth'.format(
            model_name, long_term_length, short_term_length, long_term_pred_length, hidden_size, num_layers
        )
        print("Model path:", model_path)
    else:
        model_path = './trained_models/{}_{}_{}.pth'.format(
            model_name, short_term_length, short_term_pred_length
        )
        print("Model path:", model_path)
        

    # Load pre-trained model or train a new one
    if os.path.exists(model_path):
        print(f"Loading the pre-trained {model_name} model...")
        model.load_state_dict(torch.load(model_path))
        model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    else:
        print(f"{model_name} model not found. Training a new model...")
        if 'LS_CNNLSTM' in model_name:
            train_and_evaluate3(
                model=model,
                model_name=model_name,
                train_data=(train_long, train_short),
                train_targets=(train_targets_long, train_targets_short),
                eval_data=(eval_long, eval_short),
                eval_targets=(eval_targets_long, eval_targets_short),
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience,
                oversample_eval=True
            )
        else:
            train_and_evaluate2(
                model=model,
                model_name=model_name,
                train_sequences=train_short,
                train_targets=train_targets_short,
                eval_sequences=eval_short,
                eval_targets=eval_targets_short,
                model_path=model_path,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patience=patience
            )

    return model

In [9]:
def smape(y_true, y_pred):
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape = np.mean(numerator / denominator) * 100
    return smape

# MASE 정의
def mase(y_true, y_pred, naive_forecast=None):
    mae_pred = np.mean(np.abs(y_true - y_pred))
    if naive_forecast is None:
        naive_forecast = np.roll(y_true, shift=1)
        naive_forecast[0] = y_true[0]  # Shift로 발생하는 문제 해결
    mae_naive = np.mean(np.abs(y_true - naive_forecast))
    mase = mae_pred / mae_naive
    return mase


In [13]:
def evaluate_model2(model, eval_sequences, eval_targets, model_name="", batch_size=64):
    """
    Evaluate a given model with R², SMAPE, and MASE.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    eval_dataset = torch.utils.data.TensorDataset(torch.tensor(eval_sequences, dtype=torch.float32),
                                                  torch.tensor(eval_targets, dtype=torch.float32))
    eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset, batch_size=batch_size, shuffle=False)

    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for sequences_batch, targets_batch in eval_loader:
            sequences_batch, targets_batch = sequences_batch.to(device), targets_batch.to(device)
            
            if 'Att' in model_name:
                outputs, _ = model(sequences_batch)
            else:
                outputs = model(sequences_batch)

            # 필요에 따라 squeeze() 적용
            if outputs.shape[-1] == 1:
                outputs = outputs.squeeze(-1)

            all_outputs.append(outputs.cpu().numpy())
            all_targets.append(targets_batch.cpu().numpy())

    all_outputs = np.concatenate(all_outputs, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Metrics calculation
    r2 = r2_score(all_targets, all_outputs)
    n = len(all_targets)
    k = eval_data[0].shape[-1] if is_multi_input else eval_data.shape[-1]
    adjusted_r2 = 1 - ((1 - r2) * (n - 1)) / (n - k - 1)
    smape_score = smape(all_targets, all_outputs)
    mase_score = mase(all_targets, all_outputs)

    # Print results
    print(f"R² Score: {r2:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    print(f"SMAPE: {smape_score:.2f}")
    print(f"MASE: {mase_score:.4f}")

    return {"R2": r2, "Adjusted R2": adjusted_r2, "SMAPE": smape_score, "MASE": mase_score}

In [19]:
def evaluate_model3(model, eval_data, eval_targets, model_name="", 
                    batch_size=64, oversample_eval=False):
    """
    Evaluate the model using R², SMAPE, and MASE metrics.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    eval_long, eval_short = eval_data
    eval_targets_long, eval_targets_short = eval_targets

    # Optionally oversample evaluation data
    if oversample_eval:
        eval_long, eval_short, eval_targets_long, eval_targets_short = oversample_data(
            eval_long, eval_short, eval_targets_long, eval_targets_short
        )

    # Adjust evaluation data sizes if oversample is not applied
    if not oversample_eval:
        min_eval_size = min(len(eval_long), len(eval_short))
        eval_long = eval_long[:min_eval_size]
        eval_short = eval_short[:min_eval_size]
        eval_targets_long = eval_targets_long[:min_eval_size]
        eval_targets_short = eval_targets_short[:min_eval_size]

    eval_dataset =  torch.utils.data.TensorDataset(eval_long, eval_short, eval_targets_long, eval_targets_short)
    eval_loader = DataLoader(dataset=eval_dataset, batch_size=batch_size, shuffle=False)

    all_outputs_long = []
    all_outputs_short = []
    all_targets_long = []
    all_targets_short = []

    with torch.no_grad():
        for long_batch, short_batch, long_target, short_target in eval_loader:
            long_batch, short_batch = long_batch.to(device), short_batch.to(device)
            long_target, short_target = long_target.to(device), short_target.to(device)

            if 'Att' in model_name:
                output_long, output_short, attention_weights = model(long_batch, short_batch)
            else:
                output_long, output_short = model(long_batch, short_batch)

            all_outputs_long.append(output_long.cpu().numpy())
            all_outputs_short.append(output_short.cpu().numpy())
            all_targets_long.append(long_target.cpu().numpy())
            all_targets_short.append(short_target.cpu().numpy())

    all_outputs_long = np.concatenate(all_outputs_long, axis=0)
    all_outputs_short = np.concatenate(all_outputs_short, axis=0)
    all_targets_long = np.concatenate(all_targets_long, axis=0)
    all_targets_short = np.concatenate(all_targets_short, axis=0)

    # Metrics calculation for long-term and short-term predictions
    r2_short = r2_score(all_targets_short, all_outputs_short)
    n = len(all_targets_short)
    k = eval_data[0].shape[-1]
    adjusted_r2 = 1 - ((1 - r2_short) * (n - 1)) / (n - k - 1)
    smape_short = smape(all_targets_short, all_outputs_short)
    mase_short = mase(all_targets_short, all_outputs_short)


    # Print results
    print(f"R² Score: {r2_short:.4f}")
    print(f"Adjusted R²: {adjusted_r2:.4f}")
    print(f"SMAPE: {smape_short:.2f}")
    print(f"MASE: {mase_short:.4f}")
    
    return {
        "R2 Short": r2_short,
        "Adjusted R2": adjusted_r2,
        "SMAPE Short": smape_short,
        "MASE Short": mase_short
    }


In [20]:
params = {
    'model_name' : 'LS_CNNLSTM', 
    'hidden_size' : 512, # 256
    'num_layers' : 3,  # 3
    'dropout' : 0.3,
    'num_epochs' : 150,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'patience' : 15,
}

# build model 
model = load_trained_model(params)

print("Evaluating the model...")

if 'LS_CNNLSTM' in params['model_name']:
    results = evaluate_model3(
        model=model,
        eval_data=(eval_long, eval_short),
        eval_targets=(eval_targets_long, eval_targets_short),
        model_name=params['model_name'],
        batch_size=params['batch_size'], 
        oversample_eval=True 
    )
else:
    results = evaluate_model2(
        model=model,
        eval_sequences=eval_short,
        eval_targets=eval_targets_short,
        model_name=params['model_name'],
        batch_size=params['batch_size']
    )

Model path: ./trained_models/LS_CNNLSTM_long_1460_short_720_256_512_3_ab.pth
Loading the pre-trained LS_CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_157/3271299019.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


R² Score: 0.8222
Adjusted R²: 0.8216
SMAPE: 34.95
MASE: 0.6158


In [ ]:
LS_CNNLSTM_long_1460_short_720_256_256_3.pth

R² Long: 0.5076, R² Short: 0.7703
SMAPE Long: 38.99, SMAPE Short: 37.88
MASE Long: 0.4550, MASE Short: 0.7009

LS_CNNLSTM_long_1460_short_720_256_512_3.pth
R² Long: 0.5237, R² Short: 0.8254
SMAPE Long: 36.39, SMAPE Short: 34.46
MASE Long: 0.4416, MASE Short: 0.6088

LS_CNNLSTM_long_1460_short_720_256_1024_3.pth
R² Long: 0.2760, R² Short: 0.7326
SMAPE Long: 41.78, SMAPE Short: 39.92
MASE Long: 0.5381, MASE Short: 0.7538




LS_CNNLSTM_long_1460_short_720_256_256_3_ab.pth
R² Long: 0.5081, R² Short: 0.8329
SMAPE Long: 38.46, SMAPE Short: 33.86
MASE Long: 0.4528, MASE Short: 0.5994

LS_CNNLSTM_long_1460_short_720_256_512_3_ab.pth
R² Long: 0.4195, R² Short: 0.8222
SMAPE Long: 38.95, SMAPE Short: 34.95
MASE Long: 0.4866, MASE Short: 0.6158

In [18]:
LS_CNNLSTM_Att_long_1460_short_720_256_256_3
256, 256, 3
R² Long: 0.4926, R² Short: 0.6282
SMAPE Long: 41.39, SMAPE Short: 43.75
MASE Long: 0.4741, MASE Short: 0.8927


LS_CNNLSTM_Att_long_1460_short_720_256_512_3
R² Long: 0.4417, R² Short: 0.7718
SMAPE Long: 36.16, SMAPE Short: 39.04
MASE Long: 0.4698, MASE Short: 0.7043


LS_CNNLSTM_Att_long_1460_short_720_256_1024_3.pth

R² Long: 0.4569, R² Short: 0.7043
SMAPE Long: 43.27, SMAPE Short: 42.62
MASE Long: 0.4927, MASE Short: 0.7974


##########


LS_CNNLSTM_Att_long_1460_short_720_256_256_3_ab.pth
R² Long: -0.0020, R² Short: 0.7264
SMAPE Long: 56.80, SMAPE Short: 41.01
MASE Long: 0.7358, MASE Short: 0.7686


LS_CNNLSTM_Att_long_1460_short_720_256_512_3.pth
R² Long: -0.0026, R² Short: 0.7574
SMAPE Long: 56.82, SMAPE Short: 39.78
MASE Long: 0.7354, MASE Short: 0.7247

(256, 256, 3)

In [ ]:
LS_CNNLSTM_long_1460_short_168.pth

LS_CNNLSTM (oversample_eval=True) 
- 24, 128 -> 0.52
 - 128, 256 -> 0.57

In [ ]:
_LS_CNNLSTM_Att_long_1460_short_168.pth

LS_CNNLSTM_Att (oversample_eval=True) 
- 128
R² Long: 0.4111, R² Short: 0.7501
SMAPE Long: 46.69, SMAPE Short: 37.15
MASE Long: 0.5172, MASE Short: 0.7029

In [ ]:
LS_CNNLSTM_Att_long_1460_short_168_mse.pth

LS_CNNLSTM_Att (oversample_eval=True) using mse loss 
- 256
R² Long: 0.2416, R² Short: 0.8897
SMAPE Long: 51.22, SMAPE Short: 27.54
MASE Long: 0.5858, MASE Short: 0.4563

In [ ]:
LS_CNNLSTM_Att_long_1460_short_168_mse_ab.pth

LS_CNNLSTM_Att (oversample_eval=True) using mse loss + alpha/beta 
 - 256
R² Long: 0.1336, R² Short: 0.8873
SMAPE Long: 57.46, SMAPE Short: 27.97
MASE Long: 0.6374, MASE Short: 0.4611

In [ ]:
LS_CNNLSTM_Att_long_8760_short_168.pth (365*24)

LS_CNNLSTM_Att (oversample_eval=True) using mse loss 
 - 256 
R² Long: 0.5729, R² Short: 0.9021
SMAPE Long: 48.14, SMAPE Short: 25.97
MASE Long: 0.9698, MASE Short: 0.4236

In [ ]:
LSTM, GUR, LSTP+Attention, CNN-LSTM 

Long/Short CNN-LSTM, Long/Short CNN-LSTM + attention

In [36]:
params = {
    'model_name' : 'LSTM', 
    'hidden_size' : 128, # 256
    'num_layers' : 2,  # 3
    'dropout' : 0.3,
    'num_epochs' : 150,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'patience' : 5,
}

# build model 
model = load_trained_model(params)

print("Evaluating the model...")

if 'LS_CNNLSTM' in params['model_name']:
    results = evaluate_model3(
        model=model,
        eval_data=(eval_long, eval_short),
        eval_targets=(eval_targets_long, eval_targets_short),
        model_name=params['model_name'],
        batch_size=params['batch_size'], 
        oversample_eval=True 
    )
else:
    results = evaluate_model2(
        model=model,
        eval_sequences=eval_short,
        eval_targets=eval_targets_short,
        model_name=params['model_name'],
        batch_size=params['batch_size']
    )

Model path: ./trained_models/LSTM_long_8760_short_168.pth
Loading the pre-trained LSTM model...
Evaluating the model...


/tmp/ipykernel_551697/2463606209.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
/tmp/ipykernel_551697/367026898.py:9: Use

	R² Score: 0.9669
	SMAPE: 18.59
	MASE: 0.2632


In [37]:
params = {
    'model_name' : 'GRU', 
    'hidden_size' : 128, # 256
    'num_layers' : 2,  # 3
    'dropout' : 0.3,
    'num_epochs' : 150,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'patience' : 5,
}

# build model 
model = load_trained_model(params)

print("Evaluating the model...")

if 'LS_CNNLSTM' in params['model_name']:
    results = evaluate_model3(
        model=model,
        eval_data=(eval_long, eval_short),
        eval_targets=(eval_targets_long, eval_targets_short),
        model_name=params['model_name'],
        batch_size=params['batch_size'], 
        oversample_eval=True 
    )
else:
    results = evaluate_model2(
        model=model,
        eval_sequences=eval_short,
        eval_targets=eval_targets_short,
        model_name=params['model_name'],
        batch_size=params['batch_size']
    )

Model path: ./trained_models/GRU_long_8760_short_168.pth
Loading the pre-trained GRU model...
Evaluating the model...


/tmp/ipykernel_551697/2463606209.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
/tmp/ipykernel_551697/367026898.py:9: Use

	R² Score: 0.9550
	SMAPE: 21.36
	MASE: 0.3107


In [39]:
params = {
    'model_name' : 'CNNLSTM', 
    'hidden_size' : 256, # 256
    'num_layers' : 3,  # 3
    'dropout' : 0.3,
    'num_epochs' : 150,
    'batch_size' : 16,
    'learning_rate' : 0.001,
    'patience' : 5,
}

# build model 
model = load_trained_model(params)

print("Evaluating the model...")

if 'LS_CNNLSTM' in params['model_name']:
    results = evaluate_model3(
        model=model,
        eval_data=(eval_long, eval_short),
        eval_targets=(eval_targets_long, eval_targets_short),
        model_name=params['model_name'],
        batch_size=params['batch_size'], 
        oversample_eval=True 
    )
else:
    results = evaluate_model2(
        model=model,
        eval_sequences=eval_short,
        eval_targets=eval_targets_short,
        model_name=params['model_name'],
        batch_size=params['batch_size']
    )

Model path: ./trained_models/CNNLSTM_long_8760_short_168.pth
Loading the pre-trained CNNLSTM model...
Evaluating the model...


/tmp/ipykernel_551697/2463606209.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
/tmp/ipykernel_551697/367026898.py:9: Use

	R² Score: 0.9830
	SMAPE: 13.24
	MASE: 0.1815
